In [19]:
%%writefile main.cpp
#include <iostream>
#include <cmath>
#include <iomanip>
#include <vector>
#include <string>
#include <fstream> // Required for file input/output
#include <sstream> // Required for parsing lines from file
#include <map>     // For counting statuses
#include <algorithm> // Required for std::replace


// Function to calculate f(v) = 2*c1*v - 2*c2*v^(-3)
double f_drone(double v, double c1, double c2) {
    if (v <= 1e-9) {
        return 1e12;
    }
    return 2.0 * c1 * v - (2.0 * c2 / (v * v * v));
}

// Function to calculate f'(v) = 2*c1 + 6*c2*v^(-4)
double df_drone(double v, double c1, double c2) {
    if (v <= 1e-9) {
        return 1e12;
    }
    return 2.0 * c1 + (6.0 * c2 / (v * v * v * v));
}

// Structure to hold results for each test case
struct TestCaseResult {
    double c1_param;
    double c2_param;
    double v0_initial;
    double v_optimal_numerical;
    int iterations_taken;
    double v_optimal_analytical;
    double relative_error_percent;
    std::string status_message;
    double c2_c1_ratio; // Added for plotting
};

// Newton-Raphson implementation
TestCaseResult newton_raphson_solver(double c1, double c2, double v0, double tolerance, int max_iter, bool print_iterations_detail = false) {
    double v_current = v0;
    int iter = 0;
    TestCaseResult result = {c1, c2, v0, v0, 0, 0.0, -1.0, "Max Iterations Reached", 0.0};

    if (v0 <= 0) {
        result.status_message = "Invalid Initial Guess (v0 <= 0)";
        return result;
    }
     if (c1 <= 0 || c2 <= 0) {
        result.status_message = "Invalid Parameters (c1 or c2 <= 0)";
        return result;
    }
    result.c2_c1_ratio = (c1 > 0) ? (c2 / c1) : 0.0;


    if (print_iterations_detail) {
        std::cout << "\n--- Iteration Details for c1=" << c1 << ", c2=" << c2 << ", v0=" << v0 << " ---\n";
        std::cout << std::setw(10) << "Iteration"
                  << std::setw(18) << "v_i (m/s)"
                  << std::setw(18) << "f(v_i)"
                  << std::setw(18) << "f'(v_i)"
                  << std::setw(18) << "v_{i+1} (m/s)"
                  << std::setw(20) << "|v_{i+1}-v_i|" << std::endl;
        std::cout << std::string(102, '-') << std::endl;
    }

    while (iter < max_iter) {
        double f_val = f_drone(v_current, c1, c2);
        double df_val = df_drone(v_current, c1, c2);

        if (std::abs(df_val) < 1e-10) {
            result.status_message = "Derivative Zero";
            result.v_optimal_numerical = v_current;
            result.iterations_taken = iter;
            return result;
        }

        double v_next = v_current - f_val / df_val;
        double change = std::abs(v_next - v_current);

        if (v_next <= 0) {
            result.status_message = "Non-Positive Velocity";
            result.v_optimal_numerical = v_current;
            result.iterations_taken = iter;
            return result;
        }

        if (print_iterations_detail) {
            std::cout << std::fixed << std::setprecision(8)
                      << std::setw(10) << iter
                      << std::setw(18) << v_current
                      << std::setw(18) << f_val
                      << std::setw(18) << df_val
                      << std::setw(18) << v_next
                      << std::setw(20) << change << std::endl;
        }

        if (change < tolerance) {
            result.status_message = "Converged";
            result.v_optimal_numerical = v_next;
            result.iterations_taken = iter + 1;
            return result;
        }
        v_current = v_next;
        iter++;
    }
    result.v_optimal_numerical = v_current;
    result.iterations_taken = iter;
    return result;
}

// Function to print the summary table to console
void print_summary_table(const std::vector<TestCaseResult>& results) {
    std::cout << "\n\n--- Summary Table of Newton-Raphson Results for Drone Optimization ---" << std::endl;
    std::cout << std::setw(10) << "c1"
              << std::setw(10) << "c2"
              << std::setw(15) << "v0_initial"
              << std::setw(20) << "v_optimal_num"
              << std::setw(12) << "Iterations"
              << std::setw(20) << "v_optimal_analyt"
              << std::setw(18) << "Rel.Error (%)"
              << std::setw(25) << "Status" << std::endl;
    std::cout << std::string(130, '-') << std::endl;

    for (const auto& res : results) {
        std::cout << std::fixed << std::setprecision(5)
                  << std::setw(10) << res.c1_param
                  << std::setw(10) << res.c2_param
                  << std::setw(15) << res.v0_initial
                  << std::setw(20) << res.v_optimal_numerical
                  << std::setw(12) << res.iterations_taken
                  << std::setw(20) << res.v_optimal_analytical;
        if (res.relative_error_percent >= 0) {
            std::cout << std::setw(18) << std::setprecision(5) << res.relative_error_percent;
        } else {
            std::cout << std::setw(18) << "N/A";
        }
        std::cout << std::setw(25) << res.status_message << std::endl;
    }
    std::cout << std::string(130, '-') << std::endl;
}

// Function to generate data files for Gnuplot
void generate_gnuplot_data_files(const std::vector<TestCaseResult>& all_results) {
    std::ofstream iterations_file("plot_data_iterations.dat");
    std::ofstream velocities_file("plot_data_velocities.dat");
    std::ofstream errors_file("plot_data_errors.dat");
    std::ofstream status_file("plot_data_status_summary.dat");
    std::ofstream iterations_vs_ratio_file("plot_data_iterations_vs_ratio.dat");

    std::map<std::string, int> status_counts;

    for (const auto& res : all_results) {
        status_counts[res.status_message]++;
        if (res.status_message == "Converged") {
            if (iterations_file.is_open()) {
                iterations_file << res.iterations_taken << std::endl;
            }
            if (res.v_optimal_analytical > 0) { // Ensure analytical solution is valid
                if (velocities_file.is_open()) {
                    velocities_file << res.v_optimal_analytical << " " << res.v_optimal_numerical << std::endl;
                }
                if (res.relative_error_percent >= 0) {
                     if (errors_file.is_open()) {
                        errors_file << res.relative_error_percent << std::endl;
                    }
                }
            }
            if (iterations_vs_ratio_file.is_open() && res.c1_param > 0) {
                 iterations_vs_ratio_file << (res.c2_param / res.c1_param) << " " << res.iterations_taken << std::endl;
            }
        }
    }

    if (status_file.is_open()) {
        for (const auto& pair : status_counts) {
            // Gnuplot's histogram from data file expects "label" value, so we format it.
            // Replace spaces in label with underscores for easier Gnuplot handling if needed, or quote them.
            std::string label = pair.first;
            std::replace(label.begin(), label.end(), ' ', '_'); // Optional: for simpler Gnuplot labels
            status_file << "\"" << pair.first << "\" " << pair.second << std::endl;
        }
    }

    if(iterations_file.is_open()) iterations_file.close();
    if(velocities_file.is_open()) velocities_file.close();
    if(errors_file.is_open()) errors_file.close();
    if(status_file.is_open()) status_file.close();
    if(iterations_vs_ratio_file.is_open()) iterations_vs_ratio_file.close();

    std::cout << "\nData files for Gnuplot generated:" << std::endl;
    std::cout << " - plot_data_iterations.dat" << std::endl;
    std::cout << " - plot_data_velocities.dat" << std::endl;
    std::cout << " - plot_data_errors.dat" << std::endl;
    std::cout << " - plot_data_status_summary.dat" << std::endl;
    std::cout << " - plot_data_iterations_vs_ratio.dat" << std::endl;
}

// Function to generate Gnuplot script files
void generate_gnuplot_scripts() {
    // Script for Iterations Distribution
    std::ofstream iter_gp("plot_script_iterations.gp");
    if (iter_gp.is_open()) {
        iter_gp << "set terminal pngcairo enhanced font 'Verdana,10' size 800,600\n";
        iter_gp << "set output 'plot_iterations_distribution.png'\n";
        iter_gp << "set title 'Distribution of Iterations Taken for Convergence'\n";
        iter_gp << "set xlabel 'Number of Iterations'\n";
        iter_gp << "set ylabel 'Frequency'\n";
        iter_gp << "set style fill solid 0.5 border -1\n";
        iter_gp << "set boxwidth 0.9 relative\n";
        iter_gp << "bin_width = 1 # Adjust bin width as needed\n";
        iter_gp << "bin_number(x) = floor(x/bin_width)\n";
        iter_gp << "rounded_bin_value(x) = bin_width*(bin_number(x) + 0.5)\n";
        iter_gp << "plot 'plot_data_iterations.dat' using (rounded_bin_value($1)):(1) smooth frequency with boxes title 'Iterations'\n";
        iter_gp.close();
    }

    // Script for Numerical vs Analytical Velocity
    std::ofstream vel_gp("plot_script_velocities.gp");
    if (vel_gp.is_open()) {
        vel_gp << "set terminal pngcairo enhanced font 'Verdana,10' size 800,600\n";
        vel_gp << "set output 'plot_numerical_vs_analytical_velocity.png'\n";
        vel_gp << "set title 'Numerical vs. Analytical Optimal Velocity (Converged Cases)'\n";
        vel_gp << "set xlabel 'Analytical Optimal Velocity (v_{optimal_analyt})'\n";
        vel_gp << "set ylabel 'Numerical Optimal Velocity (v_{optimal_num})'\n";
        vel_gp << "set grid\n";
        vel_gp << "plot 'plot_data_velocities.dat' using 1:2 with points pt 7 ps 0.5 title 'Data Points', \\\n";
        vel_gp << "     x title 'Ideal Match (y=x)' with lines lt 1 lc rgb 'black'\n";
        vel_gp.close();
    }

    // Script for Relative Error Distribution
    std::ofstream err_gp("plot_script_errors.gp");
    if (err_gp.is_open()) {
        err_gp << "set terminal pngcairo enhanced font 'Verdana,10' size 800,600\n";
        err_gp << "set output 'plot_relative_error_distribution.png'\n";
        err_gp << "set title 'Distribution of Relative Error (%) (Converged Cases)'\n";
        err_gp << "set xlabel 'Relative Error (%)'\n";
        err_gp << "set ylabel 'Frequency'\n";
        err_gp << "set style fill solid 0.5 border -1\n";
        err_gp << "set boxwidth 0.05 relative # Adjust based on typical error magnitudes\n";
        err_gp << "bin_width_err = 0.01 # Adjust bin width for errors\n";
        err_gp << "bin_number_err(x) = floor(x/bin_width_err)\n";
        err_gp << "rounded_bin_value_err(x) = bin_width_err*(bin_number_err(x) + 0.5)\n";
        err_gp << "plot 'plot_data_errors.dat' using (rounded_bin_value_err($1)):(1) smooth frequency with boxes title 'Relative Error'\n";
        err_gp.close();
    }

    // Script for Solver Status Summary
    std::ofstream status_gp("plot_script_status.gp");
    if (status_gp.is_open()) {
        status_gp << "set terminal pngcairo enhanced font 'Verdana,10' size 800,600\n";
        status_gp << "set output 'plot_solver_status_summary.png'\n";
        status_gp << "set title 'Summary of Solver Status'\n";
        status_gp << "set ylabel 'Number of Test Cases'\n";
        status_gp << "set style data histograms\n";
        status_gp << "set style histogram clustered gap 1\n";
        status_gp << "set style fill solid 0.5 border -1\n";
        status_gp << "set boxwidth 0.9\n";
        status_gp << "set xtics rotate by -45 scale 0\n"; // Rotate x-axis labels
        status_gp << "plot 'plot_data_status_summary.dat' using 2:xtic(1) title 'Status Count' lc rgb 'blue'\n";
        status_gp.close();
    }

    // Script for Iterations vs c2/c1 ratio
    std::ofstream iter_ratio_gp("plot_script_iterations_vs_ratio.gp");
    if (iter_ratio_gp.is_open()) {
        iter_ratio_gp << "set terminal pngcairo enhanced font 'Verdana,10' size 800,600\n";
        iter_ratio_gp << "set output 'plot_iterations_vs_c2_c1_ratio.png'\n";
        iter_ratio_gp << "set title 'Iterations vs. c2/c1 Ratio (Converged Cases)'\n";
        iter_ratio_gp << "set xlabel 'Ratio c2/c1'\n";
        iter_ratio_gp << "set ylabel 'Iterations'\n";
        iter_ratio_gp << "set logscale x\n"; // Use log scale for x-axis if ratio varies widely
        iter_ratio_gp << "set grid\n";
        iter_ratio_gp << "plot 'plot_data_iterations_vs_ratio.dat' using 1:2 with points pt 7 ps 0.5 title 'Iterations vs Ratio'\n";
        iter_ratio_gp.close();
    }


    std::cout << "\nGnuplot script files generated:" << std::endl;
    std::cout << " - plot_script_iterations.gp" << std::endl;
    std::cout << " - plot_script_velocities.gp" << std::endl;
    std::cout << " - plot_script_errors.gp" << std::endl;
    std::cout << " - plot_script_status.gp" << std::endl;
    std::cout << " - plot_script_iterations_vs_ratio.gp" << std::endl;
    std::cout << "\nTo generate plots, run Gnuplot on these scripts, e.g., 'gnuplot plot_script_iterations.gp'" << std::endl;
    std::cout << "Make sure Gnuplot is installed and in your system's PATH." << std::endl;
}


int main() {
    std::vector<TestCaseResult> all_results;
    std::string input_filename = "synthetic_data.txt"; // This file should be generated by data.cpp
    std::ifstream infile(input_filename);

    if (!infile.is_open()) {
        std::cerr << "Error: Tidak dapat membuka file input: " << input_filename << std::endl;
        std::cerr << "Pastikan file 'synthetic_data.txt' ada di direktori yang sama atau jalankan generator data (data.cpp) terlebih dahulu." << std::endl;
        return 1;
    }

    std::cout << "Membaca data dari " << input_filename << " untuk Optimasi Kecepatan Drone...\n" << std::endl;

    std::string line;
    int line_number = 0;

    while (std::getline(infile, line)) {
        line_number++;
        std::istringstream iss(line);
        double c1_in, c2_in, v0_in, tol_in;
        int max_iter_in;

        if (!(iss >> c1_in >> c2_in >> v0_in >> tol_in >> max_iter_in)) {
            std::cerr << "Error parsing baris ke-" << line_number << ": " << line << std::endl;
            std::cerr << "Format yang diharapkan: c1 c2 v0_awal epsilon iterasi_maks" << std::endl;
            continue;
        }

        TestCaseResult current_res = newton_raphson_solver(c1_in, c2_in, v0_in, tol_in, max_iter_in, false);

        if (c1_in > 0 && c2_in > 0) {
            current_res.v_optimal_analytical = std::pow(c2_in / c1_in, 0.25);
            if (current_res.status_message == "Converged" && current_res.v_optimal_analytical > 0) {
                current_res.relative_error_percent = std::abs(current_res.v_optimal_numerical - current_res.v_optimal_analytical) / current_res.v_optimal_analytical * 100.0;
            } else {
                current_res.relative_error_percent = -1.0;
            }
        } else {
            current_res.v_optimal_analytical = -1.0;
            current_res.relative_error_percent = -1.0;
        }
        all_results.push_back(current_res);
    }

    infile.close();

    if (all_results.empty()) {
        if (line_number == 0) {
             std::cout << "File " << input_filename << " kosong atau tidak ada data yang valid." << std::endl;
        } else {
             std::cout << "Tidak ada data yang berhasil diproses dari " << input_filename << "." << std::endl;
        }
    } else {
        print_summary_table(all_results);
        generate_gnuplot_data_files(all_results);
        generate_gnuplot_scripts();

        // Optional: Attempt to run Gnuplot scripts automatically
        // This requires Gnuplot to be installed and in the system's PATH.
        // Use with caution, as system calls can have security implications
        // and might not work in all environments.
        #ifdef _WIN32
            // Example for Windows
            // system("gnuplot plot_script_iterations.gp");
            // system("gnuplot plot_script_velocities.gp");
            // ... and so on for other scripts
        #else
            // Example for Linux/macOS
            // system("gnuplot plot_script_iterations.gp &"); // Run in background
            // system("gnuplot plot_script_velocities.gp &");
            // ...
        #endif
        // std::cout << "\n(Attempted to call Gnuplot automatically if enabled in code - check for .png files)" << std::endl;
    }

    return 0;
}




Overwriting main.cpp


In [20]:
%%writefile data.cpp
#include <iostream>
#include <fstream>  // Untuk operasi file
#include <vector>
#include <random>   // Untuk menghasilkan angka acak
#include <iomanip>  // Untuk std::fixed dan std::setprecision

// Struktur untuk menyimpan satu set parameter input
struct InputParameters {
    double c1;
    double c2;
    double v0_initial_guess;
    double epsilon;
    int max_iterations;
};

// Fungsi untuk menghasilkan nilai double acak dalam rentang [min, max]
double random_double(double min_val, double max_val) {
    static std::mt19937 generator(std::random_device{}()); // Mesin Angka Acak Mersenne Twister
    std::uniform_real_distribution<double> distribution(min_val, max_val);
    return distribution(generator);
}

int main() {
    int num_datasets;

    std::cout << "----------------------------------------------------" << std::endl;
    std::cout << "  Generator Data Sintetis untuk Optimasi Kecepatan Drone" << std::endl;
    std::cout << "----------------------------------------------------" << std::endl;
    std::cout << "Masukkan jumlah set data yang ingin dibuat: ";
    std::cin >> num_datasets;

    if (num_datasets <= 0) {
        std::cerr << "Jumlah set data harus positif." << std::endl;
        return 1;
    }

    std::vector<InputParameters> datasets(num_datasets);

    // Tentukan rentang untuk parameter acak
    // Anda bisa menyesuaikan rentang ini sesuai kebutuhan
    double c1_min = 0.05, c1_max = 0.5;
    double c2_min = 50.0, c2_max = 500.0;
    double v0_min = 1.0, v0_max = 20.0; // Tebakan awal kecepatan
    double epsilon_val = 1e-5; // Toleransi error bisa dibuat tetap atau diacak
    int max_iter_val = 100;    // Iterasi maksimum bisa dibuat tetap atau diacak

    for (int i = 0; i < num_datasets; ++i) {
        datasets[i].c1 = random_double(c1_min, c1_max);
        datasets[i].c2 = random_double(c2_min, c2_max);

        // Untuk tebakan awal v0, kita bisa mencoba membuatnya sedikit lebih "pintar"
        // agar dekat dengan solusi analitik v_opt = (c2/c1)^(1/4)
        // Ini hanya untuk menghasilkan tebakan awal yang lebih baik, bukan solusi sebenarnya.
        double analytical_v_approx = pow(datasets[i].c2 / datasets[i].c1, 0.25);
        // Hasilkan v0 di sekitar perkiraan analitik, misal +/- 50%
        double v0_guess_min = std::max(v0_min, analytical_v_approx * 0.5);
        double v0_guess_max = std::min(v0_max, analytical_v_approx * 1.5);
        if (v0_guess_min > v0_guess_max) { // Jika rentang tidak valid karena c1/c2 ekstrim
            datasets[i].v0_initial_guess = random_double(v0_min, v0_max); // Fallback ke rentang umum
        } else {
            datasets[i].v0_initial_guess = random_double(v0_guess_min, v0_guess_max);
        }
        // Pastikan v0 tidak nol atau negatif
        if (datasets[i].v0_initial_guess <= 1e-6) {
            datasets[i].v0_initial_guess = 1.0; // Nilai default jika terlalu kecil
        }


        datasets[i].epsilon = epsilon_val; // Menggunakan nilai epsilon yang tetap
        datasets[i].max_iterations = max_iter_val; // Menggunakan iterasi maks yang tetap
    }

    // Tulis data ke file synthetic_data.txt
    std::ofstream outfile("synthetic_data.txt");
    if (!outfile.is_open()) {
        std::cerr << "Error: Tidak dapat membuka file synthetic_data.txt untuk ditulis." << std::endl;
        return 1;
    }

    // Atur presisi untuk output file
    outfile << std::fixed << std::setprecision(8);

    for (const auto& params : datasets) {
        outfile << params.c1 << " "
                << params.c2 << " "
                << params.v0_initial_guess << " "
                << params.epsilon << " "
                << params.max_iterations << std::endl;
    }

    outfile.close();

    std::cout << "\n" << num_datasets << " set data telah berhasil dibuat dan disimpan ke synthetic_data.txt" << std::endl;
    std::cout << "Format per baris: c1 c2 v0_awal epsilon iterasi_maks" << std::endl;

    return 0;
}


Overwriting data.cpp


In [21]:
!g++ main.cpp -o main
!g++ data.cpp -o generate_data -std=c++11

In [27]:
!./generate_data
!./main

----------------------------------------------------
  Generator Data Sintetis untuk Optimasi Kecepatan Drone
----------------------------------------------------
Masukkan jumlah set data yang ingin dibuat: 100

100 set data telah berhasil dibuat dan disimpan ke synthetic_data.txt
Format per baris: c1 c2 v0_awal epsilon iterasi_maks
Membaca data dari synthetic_data.txt untuk Optimasi Kecepatan Drone...



--- Summary Table of Newton-Raphson Results for Drone Optimization ---
        c1        c2     v0_initial       v_optimal_num  Iterations    v_optimal_analyt     Rel.Error (%)                   Status
----------------------------------------------------------------------------------------------------------------------------------
   0.34989 141.00186        4.72897             4.48046           4             4.48046           0.00000                Converged
   0.22453 205.42704        6.79072             5.49976           5             5.49976           0.00000                Conver

In [25]:
!sudo apt-get install gnuplot

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  aglfn gnuplot-data gnuplot-qt libevdev2 libgudev-1.0-0 libinput-bin
  libinput10 liblua5.4-0 libmd4c0 libmtdev1 libnotify4 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5printsupport5 libqt5svg5
  libqt5widgets5 libwacom-bin libwacom-common libwacom9 libwxbase3.0-0v5
  libwxgtk3.0-gtk3-0v5 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xinput0 libxcb-xkb1
  libxkbcommon-x11-0 qt5-gtk-platformtheme qttranslations5-l10n
Suggested packages:
  gnuplot-doc gnome-shell | notification-daemon qt5-image-formats-plugins
  qtwayland5
The following NEW packages will be installed:
  aglfn gnuplot gnuplot-data gnuplot-qt libevdev2 libgudev-1.0-0 libinput-bin
  libinput10 liblua5.4-0 libmd4c0 libmtdev1 libnotify4 libqt5core5a
  libqt5dbus5 libqt5gui5 libqt5network5 libqt5printsupport5

In [33]:
!gnuplot plot_script_iterations.gp
!gnuplot plot_script_velocities.gp
!gnuplot plot_script_errors.gp
!gnuplot plot_script_status.gp
!gnuplot plot_script_iterations_vs_ratio.gp